<a href="https://colab.research.google.com/github/arnav-meduri/JCVI-Syn3A-analysis/blob/main/GenerateBoundingBoxCoordinatesCSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install opencv-python
! pip install numpy
from PIL import Image, ImageDraw
import os
import pandas as pd

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
! mkdir -p /content/Masked_Images
#orig_folder = "/content/drive/MyDrive/RCompSci/Normalized_Images/jul20/original/"
masked_folder = "/content/drive/.shortcut-targets-by-id/1Gd6Qvx67HvZr5HE48UMhRc2vhUfBl1Jh/PVO_Segmentation_Project/Processed_Images/jul29/BB_Annotated_Masked_Gray_Downsampled_ALL/"
!ls "$masked_folder"
#!cp "$masked_folder"/*.png /content/Masked_Images/

'152 OD-1.png'	'24 OS-2.png'	'2 OS-4.png'	'45 OS-3.png'	'694 OS-5.png'
'152 OD-2.png'	'24 OS-3.png'	'2 OS-5.png'	'45 OS-4.png'	'934 OD-1.png'
'152 OD-3.png'	'24 OS-4.png'	'319 OD-3.png'	'45 OS-5.png'	'934 OD-2.png'
'152 OD-4.png'	'24 OS-5.png'	'352 OS-1.png'	'480 OD-1.png'	'934 OD-3.png'
'152 OD-5.png'	'261 OD-1.png'	'352 OS-4.png'	'480 OD-3.png'	'934 OD-5.png'
'1 OS-1.png'	'261 OD-2.png'	'357 OD-1.png'	'480 OD-4.png'	'966 OD-1.png'
'1 OS-4.png'	'261 OD-3.png'	'357 OD-3.png'	'480 OD-5.png'	'966 OD-2.png'
'223 OS-1.png'	'261 OD-5.png'	'357 OD-4.png'	'483 OS-2.png'	'966 OD-3.png'
'223 OS-2.png'	'267 OS-1.png'	'399 OD-1.png'	'483 OS-5.png'	'966 OD-4.png'
'223 OS-3.png'	'267 OS-3.png'	'399 OD-2.png'	'5 OS-2.png'	'966 OD-5.png'
'223 OS-4.png'	'267 OS-4.png'	'399 OD-3.png'	'690 OS-1.png'	'996 OS-3.png'
'223 OS-5.png'	'275 OD-3.png'	'399 OD-4.png'	'694 OS-1.png'
'240 OD-4.png'	'275 OD-5.png'	'399 OD-5.png'	'694 OS-2.png'
'240 OD-5.png'	'2 OS-1.png'	'45 OS-1.png'	'694 OS-3.png'
'24 OS-1.png'	'

In [11]:
import cv2
import os
import pandas as pd

def find_white_rectangles(image_path):
    # Load the image in grayscale
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Threshold the image to separate white regions
    _, thresholded = cv2.threshold(img, 200, 255, cv2.THRESH_BINARY)

    # Find connected components (white regions) in the thresholded image
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(thresholded, connectivity=4)

    # Filter out small regions (adjust min_size as needed)
    min_size = 25  # Minimum size of a rectangle to be considered
    valid_rectangles = []

    for i in range(1, num_labels):
        if stats[i][4] >= min_size:
            # Top-left coordinate
            top_left = (stats[i][0], stats[i][1])
            # Bottom-right coordinate
            bottom_right = (stats[i][0] + stats[i][2], stats[i][1] + stats[i][3])
            valid_rectangles.append((top_left, bottom_right))

    return valid_rectangles

def main(input_dir, output_csv):
    data = []

    # Get a sorted list of image files in the input directory
    image_files = sorted([f for f in os.listdir(input_dir) if f.endswith(".png") or f.endswith(".jpg")])

    # Iterate through each sorted image file
    for filename in image_files:
        image_path = os.path.join(input_dir, filename)
        rectangles = find_white_rectangles(image_path)
        print(f'filename={filename}, count={len(rectangles)}, coord={rectangles}')
        data.append([filename, len(rectangles), rectangles])

    # Create a DataFrame and save it to a CSV file
    df = pd.DataFrame(data, columns=['File Name', 'Number of Rectangles', 'Rectangle Coordinates'])
    df.to_csv(output_csv, index=False)

if __name__ == "__main__":
    input_directory = masked_folder
    output_csv_file = "output.csv"

    main(input_directory, output_csv_file)


filename=1 OS-1.png, count=1, coord=[((280, 84), (288, 95))]
filename=1 OS-4.png, count=2, coord=[((482, 29), (492, 45)), ((9, 82), (21, 98))]
filename=152 OD-1.png, count=11, coord=[((480, 34), (489, 43)), ((433, 40), (442, 52)), ((478, 65), (485, 76)), ((487, 76), (494, 87)), ((229, 96), (238, 109)), ((1, 135), (9, 148)), ((444, 190), (450, 201)), ((416, 212), (424, 224)), ((454, 213), (461, 223)), ((482, 248), (488, 257)), ((493, 261), (498, 270))]
filename=152 OD-2.png, count=12, coord=[((320, 21), (327, 32)), ((337, 21), (347, 32)), ((275, 63), (283, 73)), ((491, 67), (502, 82)), ((19, 80), (28, 92)), ((125, 88), (131, 98)), ((470, 113), (479, 126)), ((91, 123), (97, 133)), ((452, 137), (461, 152)), ((454, 164), (463, 178)), ((468, 248), (476, 258)), ((504, 259), (512, 268))]
filename=152 OD-3.png, count=13, coord=[((4, 0), (14, 11)), ((94, 28), (101, 40)), ((278, 32), (286, 44)), ((320, 35), (331, 46)), ((219, 38), (227, 51)), ((379, 58), (387, 66)), ((91, 69), (99, 81)), ((1, 10